## Import DOCX file

In [1]:
from docx import Document

In [2]:
document = Document(r"../res/test-swiss.docx")
document

In [3]:
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)

COUNTRY: SWITZERLAND
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social interactions

Retail
Upon RS
YES       
YES
Social Contact 

Social interactions

Professionals

Pro-actively
YES
YES
Social Contact 

Social interactions

Professionals

Upon RS
YES
YES
Social Contact 

Provide contact details
Retail
Pro-actively 
YES
YES
Social Contact 

Provide contact details
Retail
Upon RS 
YES
YES
Social Contact 

Provide contact details
Professionals

Pro-actively 
YES
YES
Social Contact 

Provide contact details
Professionals

Upon RS 
YES
YES
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social inte

YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Explain account opening process and provide assistance for completion of documents and account opening settin


Onboarding formality 4, including subsequent updates
Onboarding formality 4, including subsequent updates
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Onboarding formality 4, including subsequent updates
Onboarding formality 4, including subsequent updates
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Onboarding formality 4, including subsequent updates
Onboarding formality 4, including subsequent updates
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Onboarding formality 4, including subsequent updates
Onboarding formality 4, including subsequent updates
Professionals
Upon RS
YES
NO
PROSPECTS 
PROSPECTS 
PROSPECTS 
PROSPECTS 
PROSPECTS 
PROSPECTS 
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Performance of activities
Generic Market Views

Retail
Pro-actively 
NO
NO 
Performance of activities
Generic Market Views

Retail
Upon RS 
YES
NO
Performance of activities
Generic Market Views

Pr

Performance of activities 

Market Updates/Outlook (pure factual information) 

with advisory mandate
with RSSI 
with discretionary mandate
Professionals
Pro-actively 
YES
YES
Performance of activities 

Market Updates/Outlook (pure factual information) 

with advisory mandate
with RSSI 
with discretionary mandate
Professionals
Upon RS 
YES
YES
Performance of activities 

Market Updates/Outlook (pure factual information) 

without advisory mandate
Retail 
Pro-actively 
?
?
Performance of activities 

Market Updates/Outlook (pure factual information) 

without advisory mandate
Retail 
Upon RS 
?
?
Performance of activities 

Market Updates/Outlook (pure factual information) 

without advisory mandate
Professionals
Pro-actively 
?
?
Performance of activities 

Market Updates/Outlook (pure factual information) 

without advisory mandate
Professionals
Upon RS 
?
?
Performance of activities 

Porftolio Review & Risk Review  
with advisory mandate
with RSSI
with discretionary mandate
Retail 

## Make dataframes from tables

In [4]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise

In [5]:
dfs = read_docx_tables(r"../res/test-swiss.docx")

In [6]:
dfs[1]

,PROSPECTS,PROSPECTS.1,PROSPECTS.2,PROSPECTS.3,PROSPECTS.4,PROSPECTS.5
0,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,OFFSHORE SCENARIO,ONSHORE SCENARIO
1,Social Contact \n,Social interactions\n,Retail,Pro-actively,YES,YES
2,Social Contact \n,Social interactions\n,Retail,Upon RS,YES,YES
3,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,YES,YES
4,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,YES,YES
5,Social Contact \n,Provide contact details,Retail,Pro-actively,YES,YES
6,Social Contact \n,Provide contact details,Retail,Upon RS,YES,YES
7,Social Contact \n,Provide contact details,Professionals\n,Pro-actively,YES,YES
8,Social Contact \n,Provide contact details,Professionals\n,Upon RS,YES,YES


### Make single dataframe from tables

In [7]:
import io
import csv
import pandas as pd

from copy import copy
from docx import Document


def table_to_df(tab):
    """
    Parse table from a Word Document (.docx) into Pandas DataFrame
    Parameters:
        tab:   single table object from 'document.tables'
    Return: DataFrame
    """
    
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab:
            writer.writerow(row)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)
    
    CLIENT_SCENARIO_COL = 'CLIENT SCENARIO'
    
    table = [[c.text.strip(' \n') for c in r.cells] for r in tab.rows]
    client_scenario = table.pop(0)[0]
    

    table[0].insert(0, CLIENT_SCENARIO_COL)

    for row in table[1:]:
        row.insert(0, client_scenario)
    
    # ['OFFSHORE', 'ONSHORE']
    SCENARIO_COLS = list(map(lambda x: x.split()[0], table[0][-2:]))
    
    # update header
    table[0] = table[0][:-2]
    table[0] += ['SCENARIO', 'OUTCOME']
    
    # first scenario
    outcomes = []

    for idx, row in enumerate(table[1:], start=1):
        out = row[-2:]
        outcomes.append(out)
        row = row[:-2] 
        row += [SCENARIO_COLS[0], out[0]]
        table[idx] = row
        
    # second scenario
    for row, out in zip(table[1:], outcomes):
        new_row = copy(row)
        new_row[-2] = SCENARIO_COLS[1]
        new_row[-1] = out[1]

        table.append(new_row)
    
    return read_docx_tab(table)


def read_docx_tables_v2(tables):
    """
    Parameters:
        tables: list of tables to parse 'document.tables'
    Return: DataFrame containing all tables
    """
    tab_dfs = [table_to_df(tab) for tab in tables]
    return pd.concat(tab_dfs)

In [8]:
doc = Document(r"../res/test-swiss.docx")

tab1 = table_to_df(doc.tables[-1])
tab1

,CLIENT SCENARIO,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Retail,Pro-actively,OFFSHORE,YES
1,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Retail,Upon RS,OFFSHORE,YES
2,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Professionals,Pro-actively,OFFSHORE,YES
3,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Professionals,Upon RS,OFFSHORE,YES
4,EXISTING CLIENTS,Follow-up administrative tasks,"Provide account statements, investment reports...",Retail,Pro-actively,OFFSHORE,YES
5,EXISTING CLIENTS,Follow-up administrative tasks,"Provide account statements, investment reports...",Retail,Upon RS,OFFSHORE,YES
6,EXISTING CLIENTS,Follow-up administrative tasks,"Provide account statements, investment reports...",Professionals,Pro-actively,OFFSHORE,YES
7,EXISTING CLIENTS,Follow-up administrative tasks,"Provide account statements, investment reports...",Professionals,Upon RS,OFFSHORE,YES
8,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Retail,Pro-actively,ONSHORE,YES
9,EXISTING CLIENTS,Follow-up administrative tasks,Corporate actions\nAccount closure,Retail,Upon RS,ONSHORE,YES


In [9]:
read_docx_tables_v2(doc.tables[1:-5])

,CLIENT SCENARIO,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,PROSPECTS,Social Contact,Social interactions,Retail,Pro-actively,OFFSHORE,YES
1,PROSPECTS,Social Contact,Social interactions,Retail,Upon RS,OFFSHORE,YES
2,PROSPECTS,Social Contact,Social interactions,Professionals,Pro-actively,OFFSHORE,YES
3,PROSPECTS,Social Contact,Social interactions,Professionals,Upon RS,OFFSHORE,YES
4,PROSPECTS,Social Contact,Provide contact details,Retail,Pro-actively,OFFSHORE,YES
...,...,...,...,...,...,...,...
59,EXISTING CLIENTS,Marketing of Specific Services,Corporate Finance Advisory Services,Professionals,Upon RS,ONSHORE,YES
60,EXISTING CLIENTS,Marketing of Specific Services,Wealth services,Retail,Pro-actively,ONSHORE,YES
61,EXISTING CLIENTS,Marketing of Specific Services,Wealth services,Retail,Upon RS,ONSHORE,YES
62,EXISTING CLIENTS,Marketing of Specific Services,Wealth services,Professionals,Pro-actively,ONSHORE,YES


In [10]:
table_to_df(doc.tables[-4]).head()

,CLIENT SCENARIO,EXISTING CLIENTS,EXISTING CLIENTS.1,EXISTING CLIENTS.2,EXISTING CLIENTS.3,EXISTING CLIENTS.4,SCENARIO,OUTCOME
0,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,GROUP,ACTIVITY,SUBACTIVITY,CLIENT TYPE,INTERACTION MODE,EXISTING,OFFSHORE SCENARIO
1,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Pro-actively,EXISTING,YES
2,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Upon RS,EXISTING,YES
3,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Pro-actively,EXISTING,YES
4,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Upon RS,EXISTING,YES


In [11]:
# CONTRACTUAL SETUP
table_to_df(doc.tables[-2]).head()

,CLIENT SCENARIO,GROUP,ACTIVITY,CONTRACTUAL SETUP,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,EXISTING CLIENTS,Performance of activities,Investment advice,with advisory mandate\nwith RSSI,Retail,Pro-actively,OFFSHORE,YES
1,EXISTING CLIENTS,Performance of activities,Investment advice,with advisory mandate\nwith RSSI,Retail,Upon RS,OFFSHORE,YES
2,EXISTING CLIENTS,Performance of activities,Investment advice,with advisory mandate\nwith RSSI,Professionals,Pro-actively,OFFSHORE,YES
3,EXISTING CLIENTS,Performance of activities,Investment advice,with advisory mandate\nwith RSSI,Professionals,Upon RS,OFFSHORE,YES
4,EXISTING CLIENTS,Performance of activities,Investment advice,without advisory mandate,Retail,Pro-actively,OFFSHORE,NO


In [12]:
read_docx_tables_v2(doc.tables[1:]).head()

,CLIENT SCENARIO,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME,PROSPECTS,PROSPECTS.1,PROSPECTS.2,PROSPECTS.3,PROSPECTS.4,EXISTING CLIENTS,EXISTING CLIENTS.1,EXISTING CLIENTS.2,EXISTING CLIENTS.3,EXISTING CLIENTS.4,CONTRACTUAL SETUP
0,PROSPECTS,Social Contact,Social interactions,Retail,Pro-actively,OFFSHORE,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PROSPECTS,Social Contact,Social interactions,Retail,Upon RS,OFFSHORE,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PROSPECTS,Social Contact,Social interactions,Professionals,Pro-actively,OFFSHORE,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PROSPECTS,Social Contact,Social interactions,Professionals,Upon RS,OFFSHORE,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PROSPECTS,Social Contact,Provide contact details,Retail,Pro-actively,OFFSHORE,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Assemble in app